In [ ]:

import os
import copy
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
from torch import optim
from utils import *
from modules import UNet_conditional, EMA, UNet_conditional_concat, UNet_conditional_fully_concat, UNet_conditional_fully_add, UNet_conditional_concat_with_mask, UNet_conditional_concat_with_mask_v2
import logging
from torch.utils.tensorboard import SummaryWriter

logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", level=logging.INFO, datefmt="%I:%M:%S")
"""
需要修改的地方：

"""

class Diffusion:
    def __init__(self, noise_steps=1000, beta_start=1e-4, beta_end=0.02, img_size=240, device="cuda"):
        self.noise_steps = noise_steps
        self.beta_start = beta_start
        self.beta_end = beta_end

        self.beta = self.prepare_noise_schedule().to(device)
        self.alpha = 1. - self.beta
        self.alpha_hat = torch.cumprod(self.alpha, dim=0)

        self.img_size = img_size
        self.device = device

    def prepare_noise_schedule(self):
        return torch.linspace(self.beta_start, self.beta_end, self.noise_steps)

    def noise_images(self, x, t):
        sqrt_alpha_hat = torch.sqrt(self.alpha_hat[t])[:, None, None, None]
        sqrt_one_minus_alpha_hat = torch.sqrt(1 - self.alpha_hat[t])[:, None, None, None]
        Ɛ = torch.randn_like(x)
        return sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * Ɛ, Ɛ

    def _timesteps(self, n):
        return torch.randint(low=1, high=self.noise_steps, size=(n,))

    def sample(self, model, n, labels, masks, cfg_scale=0):
        logging.info(f"Sampling {n} new images....")
        model.eval()
        with torch.no_grad():
            x = torch.randn((n, 1, self.img_size, self.img_size)).to(self.device)
            for i in tqdm(reversed(range(1, self.noise_steps)), position=0):
                t = (torch.ones(n) * i).long().to(self.device)
                # predicted_noise = model(x, t, labels)
                predicted_noise = model(x, t, labels, masks)
                if cfg_scale > 0:
                    uncond_predicted_noise = model(x, t, None)
                    predicted_noise = torch.lerp(uncond_predicted_noise, predicted_noise, cfg_scale)
                alpha = self.alpha[t][:, None, None, None]
                alpha_hat = self.alpha_hat[t][:, None, None, None]
                beta = self.beta[t][:, None, None, None]
                if i > 1:
                    noise = torch.randn_like(x)
                else:
                    noise = torch.zeros_like(x)
                x = 1 / torch.sqrt(alpha) * (x - ((1 - alpha) / (torch.sqrt(1 - alpha_hat))) * predicted_noise) + torch.sqrt(beta) * noise
        return x

def inpaint_image(original_image, generated_image, mask):
    """
    将生成的图像融合到原始图像的指定区域中。
    
    参数:
    original_image (torch.Tensor): 原始图像
    generated_image (torch.Tensor): 生成的图像
    mask (torch.Tensor): 掩码图像, 1表示需要inpaint的区域, 0表示保留原图
    
    返回:
    torch.Tensor: 输出的合成图像
    """
    # 将三个输入tensor转换到相同的设备上
    device = original_image.device
    mask = mask.to(device)
    generated_image = generated_image.to(device)
    
    # 使用掩码融合原图和生成的图像
    output_image = original_image.clone()
    # print(output_image.shape, mask.shape, generated_image.shape)
    output_image = output_image * (1 - mask) + generated_image * mask
    
    return output_image



import argparse
parser = argparse.ArgumentParser()
args, unknown = parser.parse_known_args()
args.run_name = "DDPM_conditional"
args.batch_size = 10
args.image_size = 96#这个参数有什么用，和原始图像的关系是什么
# args.dataset_path =  r"D:\ASNR-MICCAI-BraTS2023-Local-Synthesis-Challenge-Training"
args.dataset_path =  r"C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data"
args.device = "cuda"
args.lr = 3e-4
args.train = True
args.shuffle = False
device = 'cuda'
dataloader = get_data(args)
model = UNet_conditional_concat_with_mask_v2().to(device)
# model = UNet_conditional_concat().to(device)
ckpt = torch.load("./models/DDPM_conditional/ema_ckpt.pt")
model.load_state_dict(ckpt)
diffusion = Diffusion(img_size=args.image_size, device=device)
pbar = tqdm(dataloader)
images, cropped_images, masks = next(iter(pbar))
modefied_images = cropped_images
b, _, _, _ = images.shape
d_images = diffusion.sample(model, n=b, labels=modefied_images, masks=masks)

# d_images = diffusion.sample(model, n=b, labels=modefied_images)


In [ ]:
def inpaint_image(original_image, generated_image, mask):
    """
    将生成的图像融合到原始图像的指定区域中。
    
    参数:
    original_image (torch.Tensor): 原始图像
    generated_image (torch.Tensor): 生成的图像
    mask (torch.Tensor): 掩码图像, 1表示需要inpaint的区域, 0表示保留原图
    
    返回:
    torch.Tensor: 输出的合成图像
    """
    # 将三个输入tensor转换到相同的设备上
    device = original_image.device
    mask = mask.to(device)
    generated_image = generated_image.to(device)
    
    # 使用掩码融合原图和生成的图像
    output_image = original_image.clone()
    print(output_image.shape, mask.shape, generated_image.shape)
    # print(output_image.shape, mask.shape, generated_image.shape)
    reference_image = output_image * mask
    output_image = output_image * (1 - mask) + generated_image * mask
    generated_image = generated_image * mask
    
    return output_image, generated_image, reference_image
images_predict_slice, generated_image, reference_image = inpaint_image(images[:,:,:,:], d_images[:,:,:,:], masks[:,:,:,:])


In [ ]:
import torch
import matplotlib.pyplot as plt



# 选择要显示的图像索引
index = 8
images = images.cpu()
d_images_new = images_predict_slice.cpu()
dd = d_images[:,:,:,:].cpu()
cropped_images = modefied_images.cpu()
g_images = generated_image.cpu()
r_images = reference_image.cpu()
masks_clone = masks.cpu()
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
# 获取选中的图像
img = images[index, 0, :, :]
d_img = d_images_new[index, 0, :, :]
c_img = cropped_images[index, 0, :, :]
g_img = g_images[index, 0, :, :]
r_img = r_images[index, 0, :, :]
dd_img = dd[index, 0, :, :]
mask = masks_clone[index, 0, :, :]
# print(g_img)
# print(img[10,:])
if torch.any(g_img != 0):
    print("g_img 中包含非0元素")
else:
    print("g_img 中全是0")
non_zero_elements = g_img[g_img != 0]
# print(non_zero_elements)
# 使用 matplotlib 显示图像
plt.figure(figsize=(8, 8))
axes[0,0].imshow(img, cmap='gray')
axes[0,0].set_title('Ground-truth')
axes[0,1].imshow(c_img, cmap='gray')
axes[0,1].set_title('Cropped guidance')
axes[1,0].imshow(dd_img, cmap='gray')
axes[1,0].set_title('DDPM genarated')
axes[1,1].imshow(d_img, cmap='gray')
axes[1,1].set_title('Final infilled image')
plt.show()
mse = nn.MSELoss()
# loss = mse(reference_image, masks_clone)
# print(loss)

In [ ]:
d_1 = d_images_new[1, 0, :, :]
d_2 = d_images_new[2, 0, :, :]
d_3 = d_images_new[3, 0, :, :]
d_4 = d_images_new[4, 0, :, :]
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
plt.figure(figsize=(8, 8))
axes[0,0].imshow(d_1, cmap='gray')
axes[0,0].set_title('Slice 1')
axes[0,1].imshow(d_2, cmap='gray')
axes[0,1].set_title('Slice 2')
axes[1,0].imshow(d_3, cmap='gray')
axes[1,0].set_title('Slice 3')
axes[1,1].imshow(d_4, cmap='gray')
axes[1,1].set_title('Slice 4')
plt.show()

In [ ]:
def _mean_squared_error(
    target: torch.Tensor,
    prediction: torch.Tensor,
    squared: bool = True,
) -> torch.Tensor:
    """
    Computes the mean squared error between the target and prediction.

    Args:
        target (torch.Tensor): The target tensor.
        prediction (torch.Tensor): The prediction tensor.
        TODO update documentation
    """
    mse = MeanSquaredError(
        squared=squared,
    )

    return mse(preds=prediction, target=target)


def _peak_signal_noise_ratio(
    target: torch.Tensor,
    prediction: torch.Tensor,
    data_range: tuple = None,
    epsilon: float = None,
) -> torch.Tensor:
    """
    Computes the peak signal to noise ratio between the target and prediction.

    Args:
        target (torch.Tensor): The target tensor.
        prediction (torch.Tensor): The prediction tensor.
        data_range (tuple, optional): If not None, this data range (min, max) is used as enumerator instead of computing it from the given data. Defaults to None.
        epsilon (float, optional): If not None, this epsilon is added to the denominator of the fraction to avoid infinity as output. Defaults to None.
    """

    if epsilon == None:
        psnr = (
            PeakSignalNoiseRatio()
            if data_range == None
            else PeakSignalNoiseRatio(data_range=data_range[1] - data_range[0])
        )
        return psnr(preds=prediction, target=target)
    else:  # implementation of PSNR that does not give 'inf'/'nan' when 'mse==0'
        mse = _mean_squared_error(target=target, prediction=prediction)
        if data_range == None:  # compute data_range like torchmetrics if not given
            min_v = (
                0 if torch.min(target) > 0 else torch.min(target)
            )  # look at this line
            max_v = torch.max(target)
        else:
            min_v, max_v = data_range
        return 10.0 * torch.log10(((max_v - min_v) ** 2) / (mse + epsilon))


def _mean_squared_log_error(
    target: torch.Tensor,
    prediction: torch.Tensor,
) -> torch.Tensor:
    """
    Computes the mean squared log error between the target and prediction.

    Args:
        target (torch.Tensor): The target tensor.
        prediction (torch.Tensor): The prediction tensor.
    """
    mle = MeanSquaredLogError()
    return mle(preds=prediction, target=target)


def _mean_absolute_error(
    target: torch.Tensor,
    prediction: torch.Tensor,
) -> torch.Tensor:
    """
    Computes the mean absolute error between the target and prediction.

    Args:
        target (torch.Tensor): The target tensor.
        prediction (torch.Tensor): The prediction tensor.
    """
    mae = MeanAbsoluteError()
    return mae(preds=prediction, target=target)

In [ ]:
from torchmetrics.image import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from torchmetrics.regression import (
    MeanAbsoluteError,
    MeanSquaredError,
    MeanSquaredLogError,
)

def _structural_similarity_index(
    target: torch.Tensor,
    prediction: torch.Tensor,
    mask: torch.Tensor = None,
) -> torch.Tensor:
    """
    Computes the structural similarity index between the target and prediction.

    Args:
        target (torch.Tensor): The target tensor.
        prediction (torch.Tensor): The prediction tensor.
        mask (torch.Tensor, optional): The mask tensor. Defaults to None.

    Returns:
        torch.Tensor: The structural similarity index.
    """
    ssim = StructuralSimilarityIndexMeasure(return_full_image=True)
    _, ssim_idx_full_image = ssim(preds=prediction, target=target)
    mask = torch.ones_like(ssim_idx_full_image) if mask is None else mask
    # ssim_idx = None
    # mask = mask.bool()
    # print(mask.bool())
    # print(ssim_idx_full_image.shape)
    try:
        ssim_idx = ssim_idx_full_image[mask]
        # print('here')
    except Exception as e:
        print(f"Error: {e}")
        if len(ssim_idx_full_image.shape) == 0:
            ssim_idx = torch.ones_like(mask) * ssim_idx_full_image
    return ssim_idx.mean()


def image_preprocess(tensor):
    # 首先将tensor转换为numpy数组
    image = tensor.numpy()
    
    # 对numpy数组进行clip和归一化操作
    t1_clipped = np.clip(image, np.quantile(image, 0.001), np.quantile(image, 0.999))
    t1_normalized = (t1_clipped - np.min(t1_clipped)) / (np.max(t1_clipped) - np.min(t1_clipped))
    
    # 将归一化后的numpy数组转换回tensor
    normalized_tensor = torch.from_numpy(t1_normalized)
    
    return normalized_tensor


img = image_preprocess(img)
d_img = image_preprocess(d_img)
print(img.shape, d_img.shape)
img_copy = img.unsqueeze(0).unsqueeze(0)
d_img_copy = d_img.unsqueeze(0).unsqueeze(0)
mask_copy = mask.unsqueeze(0).unsqueeze(0)
mask_copy = mask_copy.bool()


ssim = _structural_similarity_index(
        target=img_copy,
        prediction=d_img_copy,
        mask=mask_copy,
    ).item()
mse = _mean_squared_error(
        target=img_copy,
        prediction=d_img_copy,
        squared=True,
    ).item()
psnr = _peak_signal_noise_ratio(
        target=img_copy,
        prediction=d_img_copy,
    ).item()
print(ssim, mse, psnr)

In [ ]:

import os
import copy
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
from torch import optim
from utils import *
# from utils import get_data_inference
from modules import UNet_conditional, EMA, UNet_conditional_concat, UNet_conditional_fully_concat, UNet_conditional_fully_add
from modules import UNet_conditional_concat_with_mask, UNet_conditional_concat_with_mask_v2, UNet_conditional_concat_Large
from modules import UNet_conditional_concat_XLarge
import logging
from torch.utils.tensorboard import SummaryWriter
# from utils import _structural_similarity_index, _peak_signal_noise_ratio, _mean_squared_error

logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", level=logging.INFO, datefmt="%I:%M:%S")
"""
需要修改的地方：

"""

class Diffusion:
    def __init__(self, noise_steps=1000, beta_start=1e-4, beta_end=0.02, img_size=240, device="cuda"):
        self.noise_steps = noise_steps
        self.beta_start = beta_start
        self.beta_end = beta_end

        self.beta = self.prepare_noise_schedule().to(device)
        self.alpha = 1. - self.beta
        self.alpha_hat = torch.cumprod(self.alpha, dim=0)

        self.img_size = img_size
        self.device = device

    def prepare_noise_schedule(self):
        return torch.linspace(self.beta_start, self.beta_end, self.noise_steps)

    def noise_images(self, x, t):
        sqrt_alpha_hat = torch.sqrt(self.alpha_hat[t])[:, None, None, None]
        sqrt_one_minus_alpha_hat = torch.sqrt(1 - self.alpha_hat[t])[:, None, None, None]
        Ɛ = torch.randn_like(x)
        return sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * Ɛ, Ɛ

    def _timesteps(self, n):
        return torch.randint(low=1, high=self.noise_steps, size=(n,))

    def sample(self, model, n, labels, masks, cfg_scale=0):
        logging.info(f"Sampling {n} new images....")
        model.eval()
        with torch.no_grad():
            x = torch.randn((n, 1, self.img_size, self.img_size)).to(self.device)
            for i in tqdm(reversed(range(1, self.noise_steps)), position=0):
                t = (torch.ones(n) * i).long().to(self.device)
                # predicted_noise = model(x, t, labels)
                print(labels.shape, masks.shape)
                predicted_noise = model(x, t, labels, masks)
                if cfg_scale > 0:
                    uncond_predicted_noise = model(x, t, None)
                    predicted_noise = torch.lerp(uncond_predicted_noise, predicted_noise, cfg_scale)
                alpha = self.alpha[t][:, None, None, None]
                alpha_hat = self.alpha_hat[t][:, None, None, None]
                beta = self.beta[t][:, None, None, None]
                if i > 1:
                    noise = torch.randn_like(x)
                else:
                    noise = torch.zeros_like(x)
                x = 1 / torch.sqrt(alpha) * (x - ((1 - alpha) / (torch.sqrt(1 - alpha_hat))) * predicted_noise) + torch.sqrt(beta) * noise
        return x

def inpaint_image(original_image, generated_image, mask):
    """
    将生成的图像融合到原始图像的指定区域中。
    
    参数:
    original_image (torch.Tensor): 原始图像
    generated_image (torch.Tensor): 生成的图像
    mask (torch.Tensor): 掩码图像, 1表示需要inpaint的区域, 0表示保留原图
    
    返回:
    torch.Tensor: 输出的合成图像
    """
    # 将三个输入tensor转换到相同的设备上
    device = original_image.device
    mask = mask.to(device)
    generated_image = generated_image.to(device)
    
    # 使用掩码融合原图和生成的图像
    output_image = original_image.clone()
    # print(output_image.shape, mask.shape, generated_image.shape)
    output_image = output_image * (1 - mask) + generated_image * mask
    
    return output_image

def inpaint_image(original_image, generated_image, mask):
    """
    将生成的图像融合到原始图像的指定区域中。
    
    参数:
    original_image (torch.Tensor): 原始图像
    generated_image (torch.Tensor): 生成的图像
    mask (torch.Tensor): 掩码图像, 1表示需要inpaint的区域, 0表示保留原图
    
    返回:
    torch.Tensor: 输出的合成图像
    """
    # 将三个输入tensor转换到相同的设备上
    device = original_image.device
    mask = mask.to(device)
    generated_image = generated_image.to(device)
    
    # 使用掩码融合原图和生成的图像
    output_image = original_image.clone()
    print(output_image.shape, mask.shape, generated_image.shape)
    # print(output_image.shape, mask.shape, generated_image.shape)
    reference_image = output_image * mask
    output_image = output_image * (1 - mask) + generated_image * mask
    generated_image = generated_image * mask
    
    return output_image, generated_image, reference_image
# images_predict_slice, generated_image, reference_image = inpaint_image(images[:,:,:,:], d_images[:,:,:,:], masks[:,:,:,:])


import argparse
parser = argparse.ArgumentParser()
args, unknown = parser.parse_known_args()
args.run_name = "DDPM_conditional"
args.batch_size = 2
args.image_size = 128#这个参数有什么用，和原始图像的关系是什么
# args.dataset_path =  r"D:\ASNR-MICCAI-BraTS2023-Local-Synthesis-Challenge-Training"
args.dataset_path =  r"C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data"
args.device = "cuda"
args.lr = 1e-4
args.train = True
args.shuffle = False
device = 'cuda'
# dataloader = get_data_inference(args)
model = UNet_conditional_concat_Large().to(device)
# model = UNet_conditional_concat().to(device)
ckpt = torch.load("./models/DDPM_conditional/ema_ckpt.pt")
model.load_state_dict(ckpt)
diffusion = Diffusion(img_size=args.image_size, device=device)


test_dataloader = get_data_inference(args)
pbar_test = tqdm(test_dataloader)
for i, (images, cropped_images, masks) in enumerate(pbar_test):
    modefied_images = cropped_images
    b, _, _, _ = images.shape
    # print(modefied_images.shape)
    masks = masks.to(torch.float)
    # print(masks.shape)
    modefied_images = modefied_images.to(device)
    d_images = diffusion.sample(model, n=b, labels=modefied_images, masks=masks)
    images_predict_slice, generated_image, reference_image = inpaint_image(images[:,:,:,:], d_images[:,:,:,:], masks[:,:,:,:])
    images = images.cpu()
    d_images_new = images_predict_slice.cpu()
    dd = d_images[:,:,:,:].cpu()
    cropped_images = modefied_images.cpu()
    masks_clone = masks.cpu()


    for index in range(b):
        z = i * args.batch_size + index
        d_img = d_images_new[index, 0, :, :]
        # subject_dir = os.path.join(args.dataset_path, f'generated_slice_{z}')
        np.savez(os.path.join(args.dataset_path, f'generated_slice_{z}.npz'),
                 image = d_img)



In [ ]:
import glob
import numpy as np
import nibabel as nib
import os
import torch.nn as nn
from tqdm import tqdm
from torch import optim
from utils import *
import cv2
subject_dir = r'C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\BraTS-GLI-01666-000'
print(subject_dir)
image_path = glob.glob(os.path.join(subject_dir, '*t1n.nii.gz'))[0]
print(image_path)
cropped_image_path = glob.glob(os.path.join(subject_dir, '*t1n-voided.nii.gz'))[0]
healthy_mask_path = glob.glob(os.path.join(subject_dir, '*healthy.nii.gz'))[0]
unhealthy_mask_path = glob.glob(os.path.join(subject_dir, '*unhealthy.nii.gz'))[0]


# 加载图像和掩膜
image = nib.load(image_path).get_fdata().astype(
                np.float32
            )
healthy_mask = nib.load(healthy_mask_path).get_fdata().astype(
                np.float32
            )
unhealthy_mask = nib.load(unhealthy_mask_path).get_fdata().astype(
                np.float32
            )
cropped_image = nib.load(cropped_image_path).get_fdata().astype(
                np.float32
            )
# image = image_preprocess(image)
mask_affine = nib.load(image_path).affine
ref_img = nib.Nifti1Image(image, mask_affine)
nib.save(ref_img, 'example_ref.nii.gz')

# print(image.shape)
# cropped_image = image_preprocess(cropped_image)
voided_img = nib.Nifti1Image(cropped_image, mask_affine)
# nib.save(voided_img, 'example_voided.nii.gz')

nonzero_coords = np.nonzero(healthy_mask)
center_x = (np.min(nonzero_coords[0]) + np.max(nonzero_coords[0])) // 2
center_y = (np.min(nonzero_coords[1]) + np.max(nonzero_coords[1])) // 2
center_z = (np.min(nonzero_coords[2]) + np.max(nonzero_coords[2])) // 2
image_shape = [240,240,155]
# 计算裁剪区域的边界
img_size = 128
crop_x1 = max(center_x - int(img_size/2), 0)
crop_x2 = min(center_x + int(img_size/2), image_shape[0])
crop_y1 = max(center_y - int(img_size/2), 0)
crop_y2 = min(center_y + int(img_size/2), image_shape[1])
# crop_z1 = max(center_z - 48, 0)
# crop_z2 = min(center_z + 48, image_shape[2])
crop_z1 = np.min(nonzero_coords[2])
crop_z2 = np.max(nonzero_coords[2])

# # 如果裁剪区域小于 96x96x96,则在另一边扩展
crop_size_x = crop_x2 - crop_x1
crop_size_y = crop_y2 - crop_y1
crop_size_z = crop_z2 - crop_z1
# #保存几何坐标信息
geometric_list = [crop_x1, crop_x2, crop_y1, crop_y2, crop_z1, crop_z2]

if crop_size_x < img_size:
    if center_x - int(img_size/2) < 0:
        crop_x1 = 0
        crop_x2 = img_size
    else:
        crop_x1 = image_shape[0] - int(img_size)
        crop_x2 = image_shape[0]

if crop_size_y < img_size:
    if center_y - int(img_size/2) < 0:
        crop_y1 = 0
        crop_y2 = img_size
    else:
        crop_y1 = image_shape[1] - int(img_size)
        crop_y2 = image_shape[1]

np_org = cropped_image
np_org_clipped = np.percentile(np_org, [0.5, 99.5])
start = np.min(np_org_clipped)
end = np.max(np_org_clipped)
width = end - start
print(start, end, width)

# 保存每个slice的数据
for z in range(44):
    subject_slice_path = 'C:/Users/DELL/Desktop/DDPM/ddpm_brats/DDPM_brain/test_data/generated_slice_' + str(z) + '.npz'
    with np.load(subject_slice_path) as data:
            # np_org = np.asarray(org_nifti.dataobj)
            

            np_redef = data['image']
            

            # normalize between ...

            clipped_image = np.clip(np_redef, 0, 1)
            
            # print()
            norm_img = (clipped_image - clipped_image.min()) / (
                clipped_image.max() - clipped_image.min()
            ) * width + start
            # kernel_size = (3, 3)  # 高斯核大小
            # sigma = 0  # 高斯核标准差,0则自动计算
            # norm_img = cv2.GaussianBlur(norm_img, kernel_size, sigma)
            # d_image = data['image']
            
    image[crop_x1:crop_x2, crop_y1:crop_y2, crop_z1+z] = norm_img
    # image[:,:, crop_z1+z] = norm_img

    # slice_adjacency_image = adjacency_image[:, :, :, z]

    # slice_unhealthy_mask = unhealthy_mask[:, :, z]
img = nib.Nifti1Image(image, mask_affine)

nib.save(img, 'example.nii.gz')



In [ ]:
from inpainting.challenge_metrics_2023 import generate_metrics
result_img = nib.load(r'C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\example.nii.gz')
result = torch.Tensor(result_img.get_fdata()).unsqueeze(0)

# Healthy mask (evaluation volume)
mask_path = r'C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\BraTS-GLI-01666-000\BraTS-GLI-01666-000-mask-healthy.nii.gz'
mask_img = nib.load(mask_path)
mask_healthy = torch.Tensor(mask_img.get_fdata()).bool().unsqueeze(0)
mask_affine = nib.load(mask_path).affine

# Reference (ground truth)
t1n_path = r'C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\BraTS-GLI-01666-000\BraTS-GLI-01666-000-t1n.nii.gz'
t1n_img = nib.load(t1n_path)
t1n = torch.Tensor(t1n_img.get_fdata()).unsqueeze(0)

# Normalization Tensor (on what basis shall be normalized? On the model input!)
t1n_voided_path = r'C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\BraTS-GLI-01666-000\BraTS-GLI-01666-000-t1n-voided.nii.gz'
t1n_voided_img = nib.load(t1n_voided_path)
t1n_voided = torch.Tensor(t1n_voided_img.get_fdata()).unsqueeze(0)

def inpaint_image(original_image, generated_image, mask):
    """
    将生成的图像融合到原始图像的指定区域中。
    
    参数:
    original_image (torch.Tensor): 原始图像
    generated_image (torch.Tensor): 生成的图像
    mask (torch.Tensor): 掩码图像, 1表示需要inpaint的区域, 0表示保留原图
    
    返回:
    torch.Tensor: 输出的合成图像
    """
    # 将三个输入tensor转换到相同的设备上
    
    # 使用掩码融合原图和生成的图像
    output_image = original_image
    # print(output_image.shape, mask.shape, generated_image.shape)
    # print(output_image.shape, mask.shape, generated_image.shape)
    reference_image = output_image * mask
    output_image = output_image * (1 - mask) + generated_image * mask
    generated_image = generated_image * mask
    
    return output_image, generated_image, reference_image
images_predict_slice, generated_image, reference_image = inpaint_image(t1n_img.get_fdata(), result_img.get_fdata(), mask_img.get_fdata())
img = nib.Nifti1Image(images_predict_slice, mask_affine)

nib.save(img, 'example.nii.gz')

# # Compute metrics
metrics_dict = generate_metrics( #expected Tensor dimension: 1 x 255 x 255 x 
    prediction=torch.Tensor(images_predict_slice).unsqueeze(0),
    target=t1n,
    mask=mask_healthy,
    normalization_tensor= t1n_voided #former: t1n * ~mask_healthy
    )

print(metrics_dict)